In [1]:
%matplotlib inline
import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
import matplotlib.pyplot as plt
import mpmath
import math

from amc_utils import *

In [2]:
def generate_synthetic_dist_det(m):
    accs = [0.75 for _ in range(m)]
    accs = np.asarray(accs)
    var = np.multiply(accs,(1.0-accs))
    mu = accs
    print("mu: ", mu)
    #mu = 2*np.array(accs) - 1
    
    ratio = np.max(np.divide(mu,var))/(np.min(np.divide(mu,var)))

    return mu, var, ratio

def get_observed_matrix_one_dep(mu,var,m):
    sig = np.diag(var)
    
    sig[1,0] = 0.1
    sig[0,1] = 0.1
    mu = np.reshape(mu,[m,1])
    
    O = sig + mu @ mu.T
    Oinv = np.linalg.inv(O)
    return O, Oinv, sig

def get_observed_matrix_half_deps(mu,var,m):
    sig = np.diag(var)
    half = math.floor(m/2)
    for i in range(half):
        sig[2*i,2*i+1] = 0.1
        sig[2*i+1,2*i] = 0.1
    mu = np.reshape(mu,[m,1])
    
    O = sig + mu @ mu.T
    Oinv = np.linalg.inv(O)
    return O, Oinv, sig

### What is alternating matrix completion?

Let's say we have $$\Sigma = O - \mu \mu^{T}$$
where $\Sigma$ is a covariance matrix and $\mu$ are the means of some random variables. 

In this problem setting, we only have access to $O$ but we want to learn $\mu$ and the structure of $\Sigma$ (and also $\Sigma$.)

Let's say that we have a guess $\Omega_{0}$ of initial dependencies in $\Sigma$ (ideally a subset of the true $\Omega$). Then we can solve the problem $$ \lVert O^{-1} + zz^{T} \rVert_{\Omega_{0}}$$ (Where we can solve for $\mu$ from $z$).

After solving for $\hat{\mu}$ we can examine $$\hat{\Sigma} = O - \hat{\mu} \hat{\mu}^{T}$$. If we look at the entries of $\hat{\Sigma}$ that are not in $\Omega_{0}$, then intuitively the largest entries will be those dependencies that we missed in our initial guess. 

We can add the entry corresponding to the largest value to our set, creating $\Omega_{1}$. Then, we repeat the process until the maximum value of an entry of the inverse covariance that is not in $\Omega_{t}$ is approximately zero.

### Create a dummy data set

In [3]:
m=6
mu, var, _ = generate_synthetic_dist_det(m)
mu_true = mu
O, O_inv, sig = get_observed_matrix_half_deps(mu, var, m)

mu:  [0.75 0.75 0.75 0.75 0.75 0.75]


In [4]:
sig_inv = np.linalg.pinv(sig)

### Structure of the true inverse covariance

In [5]:
print(sig_inv)

[[ 7.45341615 -3.97515528  0.          0.          0.          0.        ]
 [-3.97515528  7.45341615  0.          0.          0.          0.        ]
 [ 0.          0.          7.45341615 -3.97515528  0.          0.        ]
 [ 0.          0.         -3.97515528  7.45341615  0.          0.        ]
 [ 0.          0.          0.          0.          7.45341615 -3.97515528]
 [ 0.          0.          0.          0.         -3.97515528  7.45341615]]


### First iteration of AMC

In [6]:
iterative_deps_mask = [(i,i) for i in range(m)]
mu = solveMatrixCompletionWithMu(O_inv,O,iterative_deps_mask)
C_synth = O - np.outer(mu, mu)
J_synth = np.linalg.pinv(C_synth)
J_clean = copy.deepcopy(J_synth)
for i in range(m):
    for j in range(m):
        if abs(J_clean[i,j]) < 0.3:
            J_clean[i,j] = 0.0
print(J_clean)
max_val, max_ind, J_clean = find_largest(O,mu,m,iterative_deps_mask,0.2)
print("Index of largest value: ", max_ind)
print("Largest value: ", max_val)

[[ 7.73765557 -3.69091586  0.          0.          0.          0.        ]
 [-3.69091586  7.73765557  0.          0.          0.          0.        ]
 [ 0.          0.          7.73765557 -3.69091586  0.          0.        ]
 [ 0.          0.         -3.69091586  7.73765557  0.          0.        ]
 [ 0.          0.          0.          0.          7.73765557 -3.69091586]
 [ 0.          0.          0.          0.         -3.69091586  7.73765557]]
Index of largest value:  (4, 5)
Largest value:  3.69091586108999


### Second iteration of AMC

In [7]:
iterative_deps_mask.append(max_ind)
mu = solveMatrixCompletionWithMu(O_inv,O,iterative_deps_mask)
C_synth = O - np.outer(mu, mu)
J_synth = np.linalg.pinv(C_synth)
J_clean = copy.deepcopy(J_synth)
for i in range(m):
    for j in range(m):
        if abs(J_clean[i,j]) < 0.3:
            J_clean[i,j] = 0.0
print(J_clean)
max_val, max_ind, J_clean = find_largest(O,mu,m,iterative_deps_mask,0.2)
print("Index of largest value: ", max_ind)
print("Largest value: ", max_val)

[[ 7.82977315 -3.59879828  0.376357    0.376357    0.          0.        ]
 [-3.59879828  7.82977315  0.376357    0.376357    0.          0.        ]
 [ 0.376357    0.376357    7.82977315 -3.59879828  0.          0.        ]
 [ 0.376357    0.376357   -3.59879828  7.82977315  0.          0.        ]
 [ 0.          0.          0.          0.          7.45341615 -3.97515528]
 [ 0.          0.          0.          0.         -3.97515528  7.45341615]]
Index of largest value:  (5, 4)
Largest value:  3.975155279503102


### Now let's turn this into an algorithm

In [8]:
def amc(O, O_inv, mu_true, thresh=0.2, nonzeros=3):
    dim = np.shape(O)[0]
    iterative_deps_mask = samplegrid(dim,dim,nonzeros)
    try:
        C_synth = O - np.outer(mu_true, mu_true)
        J_synth = np.linalg.pinv(C_synth)
    except:
        print("Failed to invert J before loop")
        return np.zeros(np.shape(mu_true)), iterative_deps_mask
    num_iters = 0
    while(True):
        num_iters = num_iters + 1
        starttime = time.time()
        mu = solveMatrixCompletionWithMu(O_inv,O,iterative_deps_mask)
        max_val, max_ind, J_clean = find_largest(O,mu,dim,iterative_deps_mask,thresh)
        #if max_val < 1e-6: return J_distances, mu_distances, max_vals, mu, num_iters
        if max_val < 1e-6: 
            return mu, iterative_deps_mask

        iterative_deps_mask.append(max_ind)
    #return J_distances, mu_distances, max_vals, mu, num_iters
    return mu, iterative_deps_mask

In [9]:
mu, end_deps = amc(O,O_inv,mu_true)
print(end_deps)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (4, 5), (5, 4), (2, 3), (3, 2), (1, 0), (0, 1)]


### How will we incorporate this into Metal?

Incorporate as a function of the Label Model class, which takes in an observation matrix and solves for mu based on a set of dependencies. Basically, AMC allows the use of Metal in cases where the dependencies are not predetermined. Caveat: Currently only solves the binary case single-task case.